## Scraping video data using YouTube API

#### Set-up

Run the following instructions on your terminal:
``` bash
pip install google-api-python-client
pip install --upgrade google-api-python-client
```

In [1]:
from googleapiclient.discovery import build
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import json

import os
os.chdir(os.path.expanduser("../"))

from dees_package.youtube_functions import *

print("Current working directory:", os.getcwd())

Current working directory: /Users/hanbinfeng/Desktop/LSE_Data_Science/ds105a-project-dees-nuts


In [2]:
credentials_file_path = "./credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [3]:
# creating service object of the youtube version 3 API
service_youtube = build('youtube', 'v3', developerKey=credentials['youtube_api_key'])

## Getting a list of music videos so that we have a list of videos to analyse


Carry out test search using `youtube_search`

In [4]:
youtube_search_data, video_id = youtube_search(service_youtube, 2000, "official music video", "video", "US", 10)

with open('./data/yt_search_data.json', 'w') as json_file:
    json.dump(youtube_search_data, json_file, indent=4)

yt_search_df = pd.DataFrame(youtube_search_data)


1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000
1000000


In [5]:
len(video_id)

636

In [6]:
# check = pd.read_csv('./data/merged.csv')

# type(check['like_count'][0])

In [7]:
yt_search_df.to_csv('./data/search.csv')

## From the list of music videos, get statistics on each video


In [8]:
video_stats = get_stats(service_youtube, video_id)  # there is a limit on the number of video ids, can only run 50 at a time. Solution: create different lists with 50 IDs each.

#with open('./data/mv_stats_data.json', 'w') as json_file:
#     json.dump(video_stats, json_file, indent=4)

#videostats_df = pd.DataFrame(video_stats)

0 49
[{'kind': 'youtube#video', 'etag': 'MY1Xb9OiayeKjwrtY7GNJh8L9ls', 'id': 'JGwWNGJdvx8', 'statistics': {'viewCount': '6186320102', 'likeCount': '32532278', 'favoriteCount': '0', 'commentCount': '1150099'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Pop_music']}}, {'kind': 'youtube#video', 'etag': '6zepgk-FIwPJXkZIa4VmwFQBoeA', 'id': '09R8_2nJtjg', 'statistics': {'viewCount': '3996125736', 'likeCount': '15981291', 'favoriteCount': '0', 'commentCount': '420729'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Pop_music']}}, {'kind': 'youtube#video', 'etag': 'zzrkfSnwk32w7HEce-tiLT7DtTk', 'id': 'pRpeEdMmmQ0', 'statistics': {'viewCount': '3839619902', 'likeCount': '22118223', 'favoriteCount': '0', 'commentCount': '1321835'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Music_of_Latin_America', 'https://e

[{'kind': 'youtube#video', 'etag': '7unmlnSIuqMocuuC65X951oggKc', 'id': 'bKDdT_nyP54', 'statistics': {'viewCount': '1177686736', 'likeCount': '8359924', 'favoriteCount': '0', 'commentCount': '202959'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Hip_hop_music', 'https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Pop_music', 'https://en.wikipedia.org/wiki/Rhythm_and_blues', 'https://en.wikipedia.org/wiki/Soul_music']}}, {'kind': 'youtube#video', 'etag': 'Ve0slnvXihJeEGDE2jFi5xnJtxM', 'id': 'SR6iYWJxHqs', 'statistics': {'viewCount': '1176467560', 'likeCount': '5732726', 'favoriteCount': '0', 'commentCount': '303229'}, 'topicDetails': {'topicCategories': ['https://en.wikipedia.org/wiki/Music', 'https://en.wikipedia.org/wiki/Pop_music']}}, {'kind': 'youtube#video', 'etag': 'CiExYMbUujcqWZlNDM_O5aa6IPU', 'id': '4LfJnj66HVQ', 'statistics': {'viewCount': '1173990065', 'likeCount': '12314495', 'favoriteCount': '0', 'commentCount': '758926'}, 'topicDeta

KeyError: 'likeCount'

In [ ]:
video_stats_df = pd.DataFrame(video_stats)

In [ ]:
# merge the mv stats and search dataframes
searchandstats_merged_df = pd.merge(yt_search_df, video_stats_df, left_on='video_id', right_on='video_id')
searchandstats_merged_df.to_json('./data/merged.json')

searchandstats_merged_df.to_csv('./data/merged.csv')

In [ ]:
comments_df = get_comments_in_videos(service_youtube, video_id) # note that comments are disabled for some videos

TypeError: can only concatenate str (not "int") to str

In [ ]:
comments_df.head()

,video_id,comments
0,JGwWNGJdvx8,"[Old memory 😢, ❤❤❤🎉🎉, Who here in 2489😂❤, My s..."
1,09R8_2nJtjg,"[Me I see in 2024, Who's watching this in 2025..."
2,pRpeEdMmmQ0,"[nagyon jo a dalod❤🎉, ❤💥💕, Legent listen th..."
3,lp-EO5I60KA,"[Musica muito linda 😫🎶, Ed sheeran the best, 😍..."
4,2Vv-BfVoq4g,[It's song close in my heart but that's though...


In [ ]:
deets_df = get_video_details(service_youtube, video_id)

In [ ]:
deets_df.head()
final_youtube_df = pd.merge(pd.merge(searchandstats_merged_df,comments_df,left_on='video_id', right_on='video_id'), deets_df, on='video_id')
final_youtube_df.to_csv('./data/final_youtube.csv')